In [ ]:
import boto3
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import current_time
from pydantic import BaseModel
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

#### Creating connection to AWS services: Bedrock

In [ ]:
session = boto3.Session(aws_access_key_id=os.getenv("aws_access_key_id"),
                        aws_secret_access_key=os.getenv("aws_secret_access_key"),
                        region_name=os.getenv('region_name'))
bedrock_client = session.client('bedrock-runtime')

### Creating the Strands Agents to Bedrock

In [ ]:
# Creating Bedrock Model Instance
bedrock_model = BedrockModel(
    model_id='amazon.nova-lite-v1:0',
    temperature=0.3,
    top_p=0.8
)

In [ ]:
# Creating an agent using BedrockModel instance
first_agent = Agent(model=bedrock_model)

In [ ]:
class WorksModel(BaseModel):
    id: str
    doi: str
    title: str
    publication_date: str
    type: str
    related_works: list[str]